<a href="https://colab.research.google.com/github/41371120h/PL-Repo.peng/blob/main/HW2_%E7%A4%BE%E5%9C%98%E6%94%B6%E6%94%AF%E7%B4%80%E9%8C%84%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
import pandas as pd
from datetime import datetime
import gradio as gr

In [77]:
# Google Sheets 認證
creds, _ = default()
gc = gspread.authorize(creds)

# 開啟試算表
gsheets = gc.open_by_url(
    'https://docs.google.com/spreadsheets/d/1fk3PhrJJur-I1_ldDBjVcqX0UZGpvL2GUzO5XsfFs4I/edit?gid=917243672#gid=917243672'
)

# 取得或建立 worksheet
try:
    worksheet = gsheets.worksheet('收支紀錄')
except:
    worksheet.append_row(['日期', '項目名稱', '收入', '支出', '請款人', '支付方式', '是否已給錢', '結餘'])

In [78]:
def ai_analysis_and_save(df, worksheet):
    """根據 DataFrame 做 AI 分析，並存入 Google 試算表"""
    total_income = df['收入'].sum()
    total_expense = df['支出'].sum()
    balance = df['結餘'].iloc[-1] if not df.empty else 0

    # 簡單 AI 規則 (可改成用 OpenAI API)
    if total_expense > total_income:
        analysis = "⚠️ 支出大於收入，請注意財務赤字！"
    elif balance > 0 and total_income > total_expense:
        analysis = "✅ 收支狀況良好，目前結餘正向。"
    else:
        analysis = "📊 收支接近平衡，建議再觀察支出。"

    # 加入「未付款」提醒
    if '是否已給錢' in df.columns:
        unpaid_count = df[df['是否已給錢'].str.contains("否", na=False)].shape[0]
        if unpaid_count > 0:
            analysis += f" 另外，有 {unpaid_count} 筆未付款項目需跟進。"

    # 存入試算表 (建立/更新一個叫 "AI分析" 的分頁)
    try:
        ai_ws = worksheet.spreadsheet.worksheet("AI分析")
    except:
        ai_ws = worksheet.spreadsheet.add_worksheet(title="AI分析", rows=100, cols=2)
        ai_ws.append_row(["時間", "分析結果"])

    ai_ws.append_row([datetime.now().strftime("%Y-%m-%d %H:%M:%S"), analysis])
    return analysis

In [79]:
def add_record(date_str, item_str, in_or_out, amount, payer, payment_method, other_method, paid_status):
    # 如果選 "其他"，就用 other_method 當支付方式
    if payment_method == "其他" and other_method.strip() != "":
        payment_method = other_method.strip()

    # 判斷收入/支出
    if in_or_out == "收入":
        income = float(amount)
        expense = 0
    else:
        expense = float(amount)
        income = 0

    # 新增資料
    new_row = [date_str, item_str, income, expense, payer, payment_method, paid_status, ""]
    worksheet.append_row(new_row)

    # 更新 DataFrame
    records = worksheet.get_all_records()
    df = pd.DataFrame(records)
    df.columns = df.columns.str.strip()
    df['收入'] = pd.to_numeric(df.get('收入', 0), errors='coerce').fillna(0)
    df['支出'] = pd.to_numeric(df.get('支出', 0), errors='coerce').fillna(0)

    # 計算累積結餘
    df['結餘'] = (df['收入'] - df['支出']).cumsum()

    # 更新試算表結餘欄位
    cell_list = worksheet.range(f"H2:H{len(df)+1}")
    for i, cell in enumerate(cell_list):
        cell.value = int(df.iloc[i]['結餘'])
    worksheet.update_cells(cell_list)

    # 計算總結
    total_income = df['收入'].sum()
    total_expense = df['支出'].sum()
    balance = df['結餘'].iloc[-1] if not df.empty else 0

    summary = f"✅ 已新增一筆紀錄！\n\n"
    summary += f"📌 項目: {item_str}\n💰 收入: {income}\n💸 支出: {expense}\n👤 請款人: {payer}\n"
    summary += f"💳 支付方式: {payment_method}\n📅 日期: {date_str}\n\n"
    summary += "--- 收支總結 ---\n"
    summary += f"總收入: {total_income:,.0f}\n"
    summary += f"總支出: {total_expense:,.0f}\n"
    summary += f"目前結餘: {balance:,.0f}\n"

    # 未付款項目
    unpaid_msg = "\n--- 未付款項目 ---\n"
    if '是否已給錢' in df.columns:
        unpaid = df[df['是否已給錢'].str.contains("否", na=False)]
        if not unpaid.empty:
            unpaid_msg += unpaid[['日期', '項目名稱', '支出', '請款人', '支付方式']].to_string(index=False)
        else:
            unpaid_msg += "✅ 沒有未付款項目"
    else:
        unpaid_msg += "❌ 試算表沒有「是否已給錢」欄位"

    # AI 分析並存入試算表
    ai_result = ai_analysis_and_save(df, worksheet)
    return summary + "\n\n--- AI 分析 ---\n" + ai_result + "\n" + unpaid_msg

In [80]:
import gradio as gr
from datetime import datetime
import pandas as pd

# 假設 worksheet 已經用 gspread 建立並連線好
# worksheet = ...

def add_record(date_str, item_str, in_or_out, amount, payer, payment_method, other_method, paid_status):
    try:
        # 這裡是你新增紀錄的邏輯範例
        income = amount if in_or_out == "收入" else 0
        expense = amount if in_or_out == "支出" else 0
        payment = other_method if payment_method == "其他" else payment_method

        new_row = {
            "日期": date_str,
            "項目名稱": item_str,
            "收入": income,
            "支出": expense,
            "請款人": payer,
            "支付方式": payment,
            "是否已給錢": paid_status
        }

        # 新增資料到試算表（示範用 append_row）
        worksheet.append_row(list(new_row.values()))

        # 讀取全部資料並做總結
        records = worksheet.get_all_records()
        df = pd.DataFrame(records)
        df['收入'] = pd.to_numeric(df.get('收入', 0), errors='coerce').fillna(0)
        df['支出'] = pd.to_numeric(df.get('支出', 0), errors='coerce').fillna(0)
        df['結餘'] = (df['收入'] - df['支出']).cumsum()

        total_income = int(df['收入'].sum())
        total_expense = int(df['支出'].sum())
        balance = int(df['結餘'].iloc[-1]) if not df.empty else 0

        summary = f"✅ 已新增一筆紀錄！\n\n"
        summary += f"📌 項目: {item_str}\n💰 收入: {income}\n💸 支出: {expense}\n👤 請款人: {payer}\n"
        summary += f"💳 支付方式: {payment}\n📅 日期: {date_str}\n\n"
        summary += "--- 收支總結 ---\n"
        summary += f"總收入: {total_income:,}\n"
        summary += f"總支出: {total_expense:,}\n"
        summary += f"目前結餘: {balance:,}\n"

        # 未付款項目
        unpaid_msg = "\n--- 未付款項目 ---\n"
        if '是否已給錢' in df.columns:
            unpaid = df[df['是否已給錢'].str.contains("否", na=False)]
            if not unpaid.empty:
                unpaid_msg += unpaid[['日期', '項目名稱', '支出', '請款人', '支付方式']].to_string(index=False)
            else:
                unpaid_msg += "✅ 沒有未付款項目"
        else:
            unpaid_msg += "❌ 試算表沒有「是否已給錢」欄位"

        # 這裡你可以呼叫AI分析並寫入試算表的函式，示範傳回空字串
        ai_result = ""  # 如果你有ai_analysis_and_save函式，可以替換這行

        return summary + "\n\n--- AI 分析 ---\n" + ai_result + "\n" + unpaid_msg

    except Exception as e:
        return f"❌ 新增紀錄失敗：{str(e)}"


def generate_analysis_to_sheet():
    try:
        records = worksheet.get_all_records()
        df = pd.DataFrame(records)
        if df.empty:
            return "❌ 收支紀錄為空，無法分析。"

        df['收入'] = pd.to_numeric(df.get('收入', 0), errors='coerce').fillna(0)
        df['支出'] = pd.to_numeric(df.get('支出', 0), errors='coerce').fillna(0)
        df['結餘'] = (df['收入'] - df['支出']).cumsum()

        total_income = int(df['收入'].sum())
        total_expense = int(df['支出'].sum())
        balance = int(df['結餘'].iloc[-1])

        if '請款人' in df.columns:
            payer_summary = df.groupby('請款人')['支出'].sum().sort_values(ascending=False)
        else:
            payer_summary = pd.Series(dtype=float)

        if '支付方式' in df.columns:
            payment_summary = df.groupby('支付方式')['支出'].sum().sort_values(ascending=False)
        else:
            payment_summary = pd.Series(dtype=float)

        try:
            analysis_ws = worksheet.spreadsheet.worksheet("分析總結")
            worksheet.spreadsheet.del_worksheet(analysis_ws)
        except:
            pass
        analysis_ws = worksheet.spreadsheet.add_worksheet(title="分析總結", rows=100, cols=10)

        analysis_ws.update("A1", [["收支分析結果"]])
        analysis_ws.update("A2", [["總收入", total_income]])
        analysis_ws.update("A3", [["總支出", total_expense]])
        analysis_ws.update("A4", [["累積結餘", balance]])

        analysis_ws.update("A6", [["請款人排名"]])
        if not payer_summary.empty:
            payer_data = [[str(p), float(v)] for p, v in payer_summary.items()]
            analysis_ws.update("A7", payer_data)
        else:
            analysis_ws.update("A7", [["無資料", 0]])

        analysis_ws.update("C6", [["支付方式分佈"]])
        if not payment_summary.empty:
            payment_data = [[str(p), float(v)] for p, v in payment_summary.items()]
            analysis_ws.update("C7", payment_data)
        else:
            analysis_ws.update("C7", [["無資料", 0]])

        return "✅ 分析完成，結果已寫入工作表『分析總結』。"

    except Exception as e:
        return f"❌ 發生錯誤：{str(e)}"


with gr.Blocks() as demo:
    gr.Markdown("# 💵 收支管理系統（Google Sheets 版）")

    with gr.Tab("新增紀錄"):
        with gr.Row():
            date_str = gr.Textbox(label="日期 (YYYY-MM-DD)", value=datetime.today().strftime("%Y-%m-%d"))
            item_str = gr.Textbox(label="項目名稱")

        with gr.Row():
            in_or_out = gr.Radio(["收入", "支出"], label="收支類型", value="支出")
            amount = gr.Number(label="金額", value=0)

        with gr.Row():
            payer = gr.Textbox(label="請款人")
            payment_method = gr.Dropdown(
                choices=["現金", "信用卡", "Line Pay", "其他"],
                label="支付方式", value="現金"
            )
            other_method = gr.Textbox(label="若選擇其他，請輸入方式")
            paid_status = gr.Radio(["是", "否"], label="是否已給錢", value="是")

        submit_btn = gr.Button("新增紀錄")
        output = gr.Textbox(label="結果", lines=15)

        submit_btn.click(
            fn=add_record,
            inputs=[date_str, item_str, in_or_out, amount, payer, payment_method, other_method, paid_status],
            outputs=output
        )

    with gr.Tab("執行數據分析"):
        gr.Markdown("### 📊 收支數據分析與報告生成")
        gr.Markdown(
            "點擊按鈕，系統將讀取所有收支紀錄，"
            "進行 **總結餘、請款人排名、支付方式分佈** 等分析，"
            "並將結果寫入名為 **分析總結** 的工作表。"
        )
        analysis_btn = gr.Button("執行分析並寫入 Google Sheet")
        analysis_output = gr.Textbox(label="分析結果狀態", lines=5)

        analysis_btn.click(
            fn=generate_analysis_to_sheet,
            inputs=[],
            outputs=analysis_output
        )

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c5a0683bc539f946ef.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
